# Processing the data

Here we process the data to obtain a smaller file that is easier to work with.

This process is very long. It took 3-4 hours on an Intel i7-1165G7 processor at 2.80 GHz using a single core, with 64 GB of RAM (with swap space) and around 200 GB of disk space.

The file `yt_metadata_en.jsonl.gz` from Zenodo (https://zenodo.org/record/4650046) should be unzipped (~ 98 Go) in the data directory. 

In [ ]:
import csv
import ijson
import pandas as pd
from tqdm import tqdm

from helpers import count_upper_and_excl, count_neg_emojis, count_neg_words, sentiment, extract_features

In [ ]:
data_path = 'data/'

### Process the big json file to small CSVs (one for each year) and then parquet

Store the columns we need in `yt_metadata_en.jsonl` (all but `title`, `tags`, `description`) in a separate CSV file for each year

In [ ]:
# Years from 2005 to 2019 (included)

#years = [str(year) for year in range(2005, 2020)]
years = ['2019'] # We only use 2019 at the moment, for convenience.

In [ ]:
# Create CSVs with only the columns to keep (drop title, description and tags)

f = open(data_path + 'yt_metadata_en.jsonl')
videos = ijson.items(f, '', multiple_values=True)

output = {year: open(f'generated/{year}/{year}_videos_few_columns.csv', 'w') for year in years}

columns_to_drop = [
    'description', 
    'tags', 
    'title'
]
columns_to_keep = [
    'categories', 
    'channel_id', 
    'crawl_date', 
    'dislike_count', 
    'display_id', 
    'duration', 
    'like_count',
    'upload_date',
    'view_count'

]
columns = [f'"{str(col)}"' for col in columns_to_keep]

# Write the headers in neach CSV
for year in years:
    output[year].write(','.join(columns))
    output[year].write('\n')

# Put each video from the json file in the CSV with the correct year
for video in tqdm(videos):
    data_to_write = [str(video[field]) for field in columns_to_keep]
    date_video = str(video['upload_date'])
    year_video = date_video[:4]
    
    written = False
    for year in years:
        if year == year_video:
            output[year].write(','.join(data_to_write))
            output[year].write('\n')
            written = True
            break
    
print('Finished') # 9min50s
for year in years:
    output[year].close()
f.close()

In [ ]:
%%time
# Convert each CSV file to a parquet file (faster to read)
types_col = {    
    'categories': 'string', 
    'channel_id': 'string', 
    'dislike_count': 'Int32',
    'display_id': 'string', 
    'duration': 'string', 
    'like_count': 'Int64',
    'view_count': 'Int64'
}
for year in tqdm(years):
    v = pd.read_csv(
        f'generated/{year}/{year}_videos_few_columns.csv', 
        sep=',', 
        header=0, 
        dtype=types_col, 
        parse_dates=['crawl_date', 'upload_date'],
        na_values=['None'],
        engine='c'
    ) # 6min36s [[Djian: It might be faster using pyarrow csv]]
    v.to_parquet(f'generated/{year}/{year}_videos_few_columns.parquet', compression=None)

In [ ]:
df_videos = {
    year: pd.read_parquet(f'generated/{year}/{year}_videos_few_columns.parquet', engine='fastparquet') 
    for year in years
}

### Split the big json into CSV files (one for each year, keeping all columns)

In [ ]:
# Create CSVs that split the big json file into years

f = open(data_path + 'yt_metadata_en.jsonl')
videos = ijson.items(f, '', multiple_values=True)

output = {year: open(f'generated/{year}/{year}_videos.csv', 'w') for year in years}

columns_to_drop = [
    'description', 
    'tags', 
    'title'
]
columns_to_keep = [
    'categories', 
    'channel_id', 
    'crawl_date', 
    'dislike_count', 
    'display_id', 
    'duration', 
    'like_count',
    'upload_date',
    'view_count'
]

columns = columns_to_keep + columns_to_drop

writer = {year: csv.DictWriter(output[year], fieldnames=columns) for year in years}

for year in years:
    writer[year].writeheader()

# Put each video from the json file in the CSV with the correct year
for video in tqdm(videos):
    data_to_write = [str(video[field]) for field in columns_to_keep + columns_to_drop]
    date_video = str(video['upload_date'])
    year_video = date_video[:4]
    
    written = False
    for year in years:
        if year == year_video:
            writer[year].writerow(video)
    
print('Finished') # 9min50s
for year in years:
    output[year].close()
f.close()

### Extract the features

In [ ]:
features = [] # to be filled

#### Count uppercase words and exclamation marks

In [ ]:
# Count in titles in 2019
df_typography_title = extract_features(text_to_features=count_upper_and_excl, year='2019', field='title')
features.append(df_typography_title)

In [ ]:
# Count in descriptions in 2019
df_typography_desc = extract_features(text_to_features=count_upper_and_excl, year='2019', field='description')
features.append(df_typography_desc)

#### Count negative emojis

In [ ]:
# Count in titles in 2019
df_emojis_title = extract_features(text_to_features=count_neg_emojis, year='2019', field='title')
features.append(df_emojis_title)

In [ ]:
# Count in descriptions in 2019
df_emojis_desc = extract_features(text_to_features=count_neg_emojis, year='2019', field='description')
features.append(df_emojis_desc)

#### Count negative words

In [ ]:
# Count in titles in 2019
df_neg_words_title = extract_features(text_to_features=count_neg_words, year='2019', field='title') # 3min22s
features.append(df_neg_words_title)

In [ ]:
# Count in descriptions in 2019
df_neg_words_desc = extract_features(text_to_features=count_neg_words, year='2019', field='description') # 6min30s
features.append(df_neg_words_desc)

#### Compute sentiment analysis

In [ ]:
# Sentiment on titles in 2019
df_sia_title = extract_features(text_to_features=sentiment, year='2019', field='title') # 20min
features.append(df_sia_title)

In [ ]:
# Sentiment on descriptions in 2019
df_sia_desc = extract_features(text_to_features=sentiment, year='2019', field='description') # 2h51min
features.append(df_sia_desc)

#### Join the dataframes and store the results

In [ ]:
# Join all the dataframes of features
df_features = features[0]
for i in range(1, len(features)):
    df_features = df_features.join(features[i])

In [ ]:
# Read the df of videos from 2019 (without titles, tags and descriptions, to have a small file)
videos_few_cols = pd.read_parquet('generated/2019/2019_videos_few_columns.parquet', engine='fastparquet')

In [ ]:
# Join the features to the videos
videos_features = videos_few_cols.join(df_features)

In [ ]:
# Store the df videos with features to a parquet file
videos_features.to_parquet(f'generated/2019/2019_videos_Typo_Emojis_NegWords_Sentiment_title_desc.parquet', compression=None) # 3s

#### How to load the dataframe with videos and features (from 2019)

In [ ]:
# Run:
videos_features = pd.read_parquet('generated/2019/2019_videos_Typo_Emojis_NegWords_Sentiment_title_desc.parquet', engine='fastparquet')